In [1]:
from datasets import load_dataset, Audio
import torch
import torch_audiomentations
from torch.utils.data import DataLoader
import lightning as L

from transformers import BatchFeature
from transformers import SequenceFeatureExtractor
from transformers.utils import PaddingStrategy
import numpy as np 
from transformers import AutoFeatureExtractor


/home/lukas/miniconda3/envs/GADME/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Dataset

In [7]:
ds = load_dataset("DBD-research-group/gadme_v1_1", "NA_subset500", cache_dir="~/data_gadme")
extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

In [3]:
#ds.cleanup_cache_files()

{'train': 0, 'test': 0}

In [8]:
ds = ds.cast_column(
    column="audio",
    feature=Audio(
        sampling_rate=16_000,
        mono=True,
        decode=True
    )
)

In [9]:
def preprocess_function(batch):
    feature_extractor = extractor
    audio_arrays = [x["array"] for x in batch["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16_000,
        padding=True,
        max_length=16_000*1,
        truncation=True,
        return_tensors="pt"
    )
    return inputs

In [10]:
ds = ds.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=50,
    load_from_cache_file=True,
    num_proc=3,
)
ds = ds.select_columns(["input_values","ebird_code"])

ds = ds.rename_column("ebird_code", "labels")

ds.set_format("np")

split=ds["train"].train_test_split(0.1, shuffle=True, seed=1)


In [ ]:
  #cache_dir: ~/data_gadme

In [25]:
ds

DatasetDict({
    train: Dataset({
        features: ['filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'call_type', 'sex', 'lat', 'long', 'microphone', 'licence', 'source', 'local_time', 'detected_events', 'quality', 'input_values'],
        num_rows: 400
    })
    test: Dataset({
        features: ['filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'call_type', 'sex', 'lat', 'long', 'microphone', 'licence', 'source', 'local_time', 'detected_events', 'quality', 'input_values'],
        num_rows: 100
    })
})

In [16]:
ds

DatasetDict({
    train: Dataset({
        features: ['filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'call_type', 'sex', 'lat', 'long', 'microphone', 'licence', 'source', 'local_time', 'detected_events', 'quality', 'input_values'],
        num_rows: 400
    })
    test: Dataset({
        features: ['filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'call_type', 'sex', 'lat', 'long', 'microphone', 'licence', 'source', 'local_time', 'detected_events', 'quality', 'input_values'],
        num_rows: 100
    })
})

In [28]:
ds["train"][0]["audio"]

{'path': None,
 'array': array([ 0.00061898, -0.00345948, -0.00828452, ...,  0.        ,
         0.        ,  0.        ]),
 'sampling_rate': 32000}

In [29]:
ds["train"].format

{'type': None,
 'format_kwargs': {},
 'columns': ['audio',
  'filepath',
  'start_time',
  'end_time',
  'low_freq',
  'high_freq',
  'ebird_code',
  'call_type',
  'sex',
  'lat',
  'long',
  'microphone',
  'licence',
  'source',
  'local_time',
  'detected_events',
  'quality'],
 'output_all_columns': False}

### Cast Column

In [30]:
ds = ds.cast_column(
            column="audio",
            feature=Audio(
                sampling_rate=32_000,
                mono=True,
                decode=True
            ))

In [31]:
ds["train"][0]["audio"]

{'path': None,
 'array': array([ 0.00061898, -0.00345948, -0.00828452, ...,  0.        ,
         0.        ,  0.        ]),
 'sampling_rate': 32000}

### Map Dataset

In [32]:
class CustomFeatureExtractor(SequenceFeatureExtractor):
    model_input_names = ["input_values", "attention_mask"]

    def __init__(
        self,
        feature_size=1,
        sampling_rate=32_000,
        padding_value=0.0,
        return_attention_mask=False,
        do_normalize=True,
        **kwargs
    ):
        # initialize sequencefeatureextractor
        super().__init__(feature_size=feature_size, sampling_rate=sampling_rate, padding_value=padding_value)
        self.return_attention_mask = return_attention_mask
        self.do_normalize = do_normalize
    
    @staticmethod
    def normalize(
        input_values,
        attention_mask,
        padding_value=0.0
    ):
        if attention_mask is not None:
            attention_mask = np.array(attention_mask, np.int32)
            normed_input_values = []

            for vector, length in zip(input_values, attention_mask.sum(-1)):
                normed_slice = (vector - vector[:length].mean()) / np.sqrt(vector[:length].var() + 1e-7)
                if length < normed_slice.shape[0]:
                    normed_slice[length:] = padding_value

                normed_input_values.append(normed_slice)
        else:
            normed_input_values = [(x-x.mean()) / np.sqrt(x.var() + 1e-7) for x in input_values]
        return normed_input_values
    
    def __call__(
        self, 
        raw_audio,
        padding = False, 
        max_length = None,
        truncation = False, 
        return_tensors = None,
        sampling_rate = None,
        return_attention_mask = None,
        **kwargs
    ) -> BatchFeature:
        
        # control/check sampling rate 
        if self.sampling_rate is not None: 
            if sampling_rate != self.sampling_rate:
                raise ValueError(
                    f"The model corresponding to this feature extractor: {self} was trained using a sampling rate of"
                    f"{self.sampling_rate}. Make sure that the provided `raw_audio`input was sampled with"
                    f"{self.sampling_rate} and not {sampling_rate}."
                )
        else:
            print( "It is strongly recommended to pass the ``sampling_rate`` argument to this function. \
                    Failing to do so can result in silent errors that might be hard to debug.")
        # check batch input
        is_batched_numpy = isinstance(raw_audio, np.ndarray) and len(raw_audio.shape) > 1
        is_batched = is_batched_numpy or (
            isinstance(raw_audio, (list, tuple)) and (isinstance(raw_audio[0], (np.ndarray, tuple, list)))
        )

        if not is_batched:
            raw_audio = [raw_audio]

        encoded_inputs = BatchFeature({"input_values": raw_audio})

        padded_inputs = self.pad(
            encoded_inputs,
            padding=padding,
            max_length=max_length,
            return_attention_mask=return_attention_mask,
            truncation=truncation
        )

        #convert into format 
        input_values = padded_inputs["input_values"]
        if not isinstance(input_values[0], np.ndarray):
            padded_inputs["input_values"] = [np.asarray(array, dtype=np.float32) for array in input_values]
        elif (
            not isinstance(input_values, np.ndarray)
            and isinstance(input_values[0], np.ndarray)
            and input_values[0].dtype is np.dtype(np.float64)
        ):
            padded_inputs["input_values"] = [array.astype(np.float32) for array in input_values]
        elif isinstance(input_values, np.ndarray) and input_values.dtype is np.dtype(np.float64):
            padded_inputs["input_values"] = input_values.astype(np.float32)
        # return_to_tensors comes from: transformers/src/transformers/feature_extraction_utils.py
        if return_tensors is not None:
            padded_inputs = padded_inputs.convert_to_tensors(return_tensors)
        
        attention_mask = padded_inputs.get("attention_mask")
        if attention_mask is not None: 
            padded_inputs["attention_mask"] = [np.asarray(array, dtype=np.int32) for array in attention_mask]
        
        if self.do_normalize:
            attention_mask = (
                attention_mask
                if self._get_padding_strategies(padding, max_length=max_length) is not PaddingStrategy.DO_NOT_PAD
                else None
            )
            padded_inputs["input_values"] = self.normalize(
                padded_inputs["input_values"], attention_mask=attention_mask, padding_value=self.padding_value
            )
  
        return padded_inputs

In [33]:
feature_extractor = CustomFeatureExtractor()

In [34]:
def preprocess_function(samples):
    audio_arrays = [x["array"] for x in samples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        padding=True,
        max_length=32_000*1,
        truncation=True,
        return_tensors="pt"
    )
    return inputs

In [35]:
ds = ds.map(
    preprocess_function,
    remove_columns=["audio"],
    batched=True,
    batch_size=500,
    load_from_cache_file=True,
    num_proc=4
)

In [36]:
ds["train"].format

{'type': None,
 'format_kwargs': {},
 'columns': ['filepath',
  'start_time',
  'end_time',
  'low_freq',
  'high_freq',
  'ebird_code',
  'call_type',
  'sex',
  'lat',
  'long',
  'microphone',
  'licence',
  'source',
  'local_time',
  'detected_events',
  'quality',
  'input_values'],
 'output_all_columns': False}

In [39]:
ds.set_format("torch")

In [41]:
ds["train"].format

{'type': 'torch',
 'format_kwargs': {},
 'columns': ['filepath',
  'start_time',
  'end_time',
  'low_freq',
  'high_freq',
  'ebird_code',
  'call_type',
  'sex',
  'lat',
  'long',
  'microphone',
  'licence',
  'source',
  'local_time',
  'detected_events',
  'quality',
  'input_values'],
 'output_all_columns': False}